In [ ]:
get_ipython().profile_dir.startup_dir
test 2

In [ ]:
"""from IPython.core.display import HTML, display
#http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
display(HTML('''
<script>
  jQuery(document).ready(function($) {

  $(window).load(function(){
    $('#preloader').fadeOut('slow',function(){$(this).remove();});
  });

  });
</script>

<style type="text/css">
  div#preloader { position: fixed;
      left: 0;
      top: 0;
      z-index: 999;
      width: 100%;
      height: 100%;
      overflow: visible;
      background: #fff url('http://preloaders.net/preloaders/720/Moving%20line.gif') no-repeat center center;
  }

</style>

<div id="preloader"></div>'''))"""

In [ ]:
You can use %time to time for a single run or %timeit to time it a lot of time and get the average and standard deviation. So this is useful for these simple function, how about function that calling other another function?
q

Here comes %prun, I create a dummy function that call fib1() a lot of time. You can see the loop did spend some time but most of the time is spent on fib1(

In [ ]:
from IPython.core.display import HTML, display
#http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
display(HTML('''
<script>
  function code_toggle() {
    if (code_shown) {
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
'''))

In [ ]:
from ipywidgets import widgets
text = widgets.Text(description="a=")
display(text)

text2 = widgets.Text(description="b=")
display(text2)


def handel_submit(sender):
    print(sender.value)


text.on_submit(handel_submit)
text2.on_submit(handel_submit)

In [ ]:
import math
import ipympl
import matplotlib.pyplot as plt
%matplotlib notebook

# %matplotlib ipympl
# %matplotlib notebook
# https://scipy-cookbook.readthedocs.io/items/Matplotlib_Interactive_Plotting.html


class AnnoteFinder(object):
    """callback for matplotlib to display an annotation when points are
    clicked on.  The point which is closest to the click and within
    xtol and ytol is identified.

    Register this function like this:

    fig, ax = plt.subplots()
    ax.scatter(x, y)
    af =  AnnoteFinder(x, y, ax=ax)
    fig.canvas.mpl_connect('button_press_event', af)

    Attributes:
        data (list): (x,y,anotation) data.
        links (list): ToDo.
        points (list): selected points.
        clicks (list): clicked points.
        drawnAnnotations(list): points that have been drawn and annotated.
    """

    def __init__(self, xdata, ydata, annotes=None, ax=None, xtol=None, ytol=None):
        """ 
        Args:
        xdata (list): ploted x data.
        ydata (list): ploted y data.
        annotes (:obj:'list', optional): list same length as xdata and y data of anotation text.
        ax (:obj:'mplax', optional): matplot lib axis to plot on.
        xtol (:obj:`int`, optional): tolerance for finding x from click.
        ytol (:obj:`int`, optional): tolerance for finding y from click.
        """
        # Value checking, create self.data:
        if len(ydata) != len(xdata):
            raise ValueError('xdata and ydata must be same lenght')
        if annotes:
            if len(annotes) != len(xdata):
                raise ValueError(
                    'annotes must be same lenght as xdata and ydata')
            self.data = list(zip(xdata, ydata, annotes))
        else:
            annotes = [''] * len(xdata)
            self.data = list(zip(xdata, ydata, annotes))

        # x tolerance
        if xtol is None:
            xtol = ((max(xdata) - min(xdata))/float(len(xdata)))/2
        # y tolerance
        if ytol is None:
            ytol = ((max(ydata) - min(ydata))/float(len(ydata)))/2
        self.xtol = xtol
        self.ytol = ytol
        # axis to plot on
        if ax is None:
            self.ax = plt.gca()
        else:
            self.ax = ax
        # init some things
        self.drawnAnnotations = {}
        self.links = []
        self.points = []
        self.clicks = []

    def distance(self, x1, x2, y1, y2):
        """
        return the distance between two points.
        unused!!
        """
        return(math.sqrt((x1 - x2)**2 + (y1 - y2)**2))

    def __call__(self, event):

        if event.inaxes:

            clickX = event.xdata
            clickY = event.ydata
            # self.clicks.append([clickX, clickY])
            if (self.ax is None) or (self.ax is event.inaxes):
                annotes = []
                # print(event.xdata, event.ydata)
                for ii, (x, y, a) in enumerate(self.data):
                    if ((clickX-self.xtol < x < clickX+self.xtol) and
                            (clickY-self.ytol < y < clickY+self.ytol)):
                        annotes.append((ii, x, y, a))
                if annotes:
                    # annotes.sort()
                    #ind, x, y, annote = annotes[0]
                    for ann in annotes:
                        ind, x, y, annote = ann
                        self.drawAnnote(event.inaxes, ind, x, y, annote)
                        for l in self.links:
                            l.drawSpecificAnnote(annote)

    def drawAnnote(self, ax, ind, x, y, annote):
        """
        Draw the annotation on the plot
        """
        # remove if already there
        if ind in self.drawnAnnotations:
            markers = self.drawnAnnotations[ind]
            # for m in markers:
            #m.set_visible(not m.get_visible())
            if markers[0]:
                markers[0].remove()  # text
            markers[1].remove()  # markers
            ax.figure.canvas.draw_idle()
            # delete from drawnAnnotations and points
            del self.drawnAnnotations[ind]
            self.points.remove([ind, x, y])
        else:
            if annote:
                t = ax.text(x, y, " - %s" % (annote),)
            else:
                t = None
            m = ax.scatter([x], [y], marker='d', c='r', zorder=100)
            self.points.append([ind, x, y])
            self.drawnAnnotations[ind] = (t, m)
            ax.figure.canvas.draw_idle()

    def drawSpecificAnnote(self, annote):
        annotesToDraw = [(x, y, a) for x, y, a in self.data if a == annote]
        for x, y, a in annotesToDraw:
            self.drawAnnote(self.ax, x, y, a)

In [ ]:
from IPython.display import display
import ipywidgets as widgets
x = range(10)
y = range(10)
annotes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

fig, ax = plt.subplots()
ax.scatter(x, y)
af = AnnoteFinder(x, y, annotes, ax=ax)
fig.canvas.mpl_connect('button_press_event', af)

button = widgets.Button(description="Click Me!")
display(button)


def on_button_clicked(b):
    print(af.clicks)
    print(af.drawnAnnotations)


button.on_click(on_button_clicked)


int_range = widgets.IntSlider()
display(int_range)


def on_value_change(change):
    print(change['new'])


int_range.observe(on_value_change, names='value')

In [ ]:
"""Setup for Sci Trim Calculations:
Please only edit in designated area,
Will return found file name and list of all colums in file (use to select target_param)
"""
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from fpy.file_funcs import *
from fpy.trim_funcs import *
from fpy.wafer_tools import plot_wafermap

#from fpy.basic_tools import plot_n_fit

import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from numpy import mgrid, meshgrid
import csv
from functools import partial

# Edit These!! ################################################################
freq          = "4000"
wafer         = 8270
d8_num        = "0228"
test_step     = "ResonatorMap"  
#test_step     = "WaferMap"

# Leave these as "" to auto find 
test_spec     = ""  # "m2-d8", "d8", "m3-d8",  "d8", or "dwg-"
target_param  = "ResFreq"
#target_param  = "ResFreq"  # 'ResFreq(S11)(1)', 'Vswr(-3-0)(S11)(1)', 'CF10(S21)(1)'
file_num      = -1 # "08", -1 finds most recient
comp_fn       = -2  # "06", -2 finds second most recient

# Set to None if not using
trim_to_freq = 3930
trim_to_thick = None

# rate eq
# removal[nm] = a[nm/s] * (shift[MHz] * b[s/MHz] + c[s])
a = 2.1
b = 0.1214
c = 0.1994

# Don't edit past here!! #######################################################

def rate_func_base(shift=None, a=None, b=None, c=None):

    """thickness_to_trim[nm] = rate_func(trim_to_freq - actual_freq)
    Standard form is:
        rate = (shift+X)/Y
        return (rate*z)/10
    Must return a value in nm,
    If trimming to vwsr where shift = (actual-trim_to) just make shift -: rate = (-shift...)
    """
    removal = a*(shift*b + c)
    return removal 

rate_func = partial(rate_func_base, a=a, b=b, c=c)

base_path = find_base_path()
full_path = find_test_path(base_path, str(freq), str(wafer), test_step, d8_num, test_spec)
path = os.path.join(full_path, get_filename(full_path, file_num))
print(path)
test_s = read_waferview(path)
"""Show list of all columns"""
print("\n Use this list to detrimine what to put in 'target_param':")
display(list(test_s.dataframe))
target_param = get_target_param(target_param, test_step, test_s)

# get test to compair too and ibe
target = float(trim_to_freq)
comp_fn = get_filename(full_path, comp_fn)
if comp_fn:
    comp_path = os.path.join(full_path, comp_fn)
    test_comp_s = read_waferview(comp_path)
    ibe_s = read_ibe(comp_path)
    if ibe_s.header:
        target = float(ibe_s.header['target'][0:-4])
    test_s.dataframe['target_shift'] = target - test_comp_s.dataframe[target_param]
    test_s.dataframe['actual_shift'] = test_s.dataframe[target_param] - test_comp_s.dataframe[target_param]
    test_s.dataframe['diff_shift'] = test_s.dataframe['target_shift'] - test_s.dataframe['actual_shift']

print('target:', target)
#"""Show stats for all columns"""
#stats = test_s.dataframe.describe()
#display(stats)

In [ ]:
def plot_scatter(xdata, ydata, xlabel, ylabel, ax):
    """plots and fits with a 1d line
    returns: ax"""

    # Axis Lables
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)

    # make the scatter plot
    ax.plot(xdata, ydata, linestyle='', marker='o')
    
    return ax
    
class FitLinear(object):
    """Fit line to x, y data and plot on matplotlib ax.
    
    Register this function like this:
    
    fig, ax = plt.subplots()
    ax.scatter(x, y)
    fit = FitLinear(x, y, ax=ax)

    Attributes:
        self.x
        self.y
        self.slope
        self.intercept
        self.rsqd
        self.fit
    """

    def __init__(self, xdata, ydata, ax=None):
        """ 
        Args:
        xdata (list): ploted x data.
        ydata (list): ploted y data.
        ax (:obj:'mplax', optional): matplot lib axis to plot on.
        fit (:obj:'mplax', optional): List pf mlp objects (line, yerrUpper, yerrLower, text).
        """
        # Value checking, create self.data:
        if len(ydata) != len(xdata):
                raise ValueError('xdata and ydata must be same lenght')
        self.x = xdata
        self.y = ydata
        # axis to plot on
        if ax is None:
            self.ax = plt.gca()
        else:
            self.ax = ax
        # get fit data
        self.slope, self.intercept, self.rsqd = self.get_fit(self.x, self.y)
        # plot it
        self.fit = self._plot_fit()
    
    def update_fit(self, xnew, ynew):
        """Updates plot with new data"""
        self.x = xnew
        self.y = ynew
        # get fit data
        self.slope, self.intercept, self.rsqd = self.get_fit(self.x, self.y)
        # get data for ploting
        xl, yl, xs, yerrLower, yerrUpper, eq_txt = self._get_fit_data()
        
        # Update data
        self.fit[3].set_text(eq_txt)
        # line
        self.fit[0].set_xdata(xl)
        self.fit[0].set_ydata(yl)
        # l_yerr
        self.fit[1].set_xdata(xs)
        self.fit[1].set_ydata(yerrLower)
        # u_yerr
        self.fit[2].set_xdata(xs)  
        self.fit[2].set_ydata(yerrUpper)
        
        # redraw
        self.ax.figure.canvas.draw_idle()
        #ax.relim()
        #ax.autoscale_view()
        
    def get_fit(self, xdata, ydata):
        """determine best fit line and fit parameters"""
        # get Fit!!!
        par = np.polyfit(xdata, ydata, 1, full=True)
        # get line eq
        slope = par[0][0]
        intercept = par[0][1]
        # coefficient of determination
        variance = np.var(ydata)
        residuals = np.var([(slope*xx + intercept - yy)  for xx, yy in zip(xdata, ydata)])
        rsqd = np.round(1-residuals/variance, decimals=2)
        return (slope, intercept, rsqd)
        
    def _get_fit_data(self):
        """gets extra data for plotting (line, l_yerr, u_yerr, txt)"""
        # get line
        xl = [min(self.x), max(self.x)]
        yl = [self.slope*xx + self.intercept  for xx in xl]
        # get error bounds
        yerr = [abs(self.slope*xx + self.intercept - yy)  for xx,yy in zip(self.x, self.y)]
        par = np.polyfit(self.x, yerr, 2, full=True)
        # sort the data (makes plot clean)
        reorder = sorted(range(len(self.x)), key = lambda ii: self.x[ii])
        xs = [self.x[ii] for ii in reorder]
        ys = [self.x[ii] for ii in reorder]
        # error lines
        yerrUpper = [(xx*self.slope+self.intercept)+(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx, yy in zip(xs, ys)]
        yerrLower = [(xx*self.slope+self.intercept)-(par[0][0]*xx**2 + par[0][1]*xx + par[0][2]) for xx, yy in zip(xs, ys)]
        # text to add to plot
        eq_txt = 'y=%0.2f$x + %0.2f$ \n$R^2 = %0.2f$' % (self.slope, self.intercept, self.rsqd)

        return (xl, yl, xs, yerrLower, yerrUpper, eq_txt)
    
    def _plot_fit(self):
        # get data for plotting:
        xl, yl, xs, yerrLower, yerrUpper, eq_txt = self._get_fit_data()
        # plot All
        txt = plt.text(min(self.x),.9*max(self.y),eq_txt, fontsize=30)
        line, = ax.plot(xl, yl, '-r')
        l_yerr, = ax.plot(xs, yerrLower, '--r')
        u_yerr, = ax.plot(xs, yerrUpper, '--r')
        # save plot objects
        fit = [line, l_yerr, u_yerr, txt]
        return fit


In [ ]:
import copy

"""Plot change"""
df = test_s.dataframe[['target_shift', 'actual_shift']].copy()
x_nm = 'target_shift'
y_nm = 'actual_shift'

#remove < 0
dfc = filt_positive(df, [0, 1])
print('Total removed < 0:', df.shape[0] - dfc.shape[0], 'of', df.shape[0])
dfd = filt_3sigma(dfc, [0, 1])
print('Total removed > 3\u03c3:', dfc.shape[0] - dfd.shape[0], 'of', df.shape[0])
xx = dfd[x_nm]
yy = dfd[y_nm]

# extra filtering
#filt = xx > 2
#filt = [a or b for a, b in zip(yy > 10, yy < 5)]
#filt = yy < 6
#xx = xx[filt].reset_index(drop=True)
#yy = yy[filt].reset_index(drop=True)
#################

a = float(ibe_s.header['a'])
b = float(ibe_s.header['b'])
c = float(ibe_s.header['c'])

print("a =",a ,"\nb =",b ,"\nc =",c ,'\n')
fig, ax = plt.subplots(figsize=(7,7))
ax = plot_scatter(xx, yy, x_nm, y_nm, ax)

af =  AnnoteFinder(xx, yy, ax=ax)
fig.canvas.mpl_connect('button_press_event', af)

fit = FitLinear(xx, yy, ax=ax)
m = fit.slope
n = fit.intercept
display('Please Select the points to remove from analysis then refit:')
button = widgets.Button(description="Fit")
display(button)

def on_button_clicked(b):
    p_sort = copy.deepcopy(af.points)
    xa = copy.deepcopy(xx)
    ya = copy.deepcopy(yy)
    p_sort.sort(key=lambda a: a[0], reverse=True)
    for point in p_sort:
        xa.pop(point[0])
        ya.pop(point[0])
        
    xa = xa.reset_index(drop=True)
    ya = ya.reset_index(drop=True)
    #print('before', fit.slope, fit.intercept)
    fit.update_fit(xa, ya)
    #print('After', fit.slope, fit.intercept)
    #print(af.drawnAnnotations)

button.on_click(on_button_clicked)



"""take initial eq and match new removal form previous run"""
# removal[nm] = a[nm/s] * (shift[MHz] * b[s/MHz] + c[s])
# actual vs target line fite eq y=mx+n
b2 = b/m # + 1/m/a
if n > 0:
    c2 = c - n*b2
else:
    c2 = c - n/a*b2
print('\nNew Eq:')
print('a =', a)
print('b =', round(b2, 4))
print('c =', round(c2, 4))

In [ ]:

print(get_precision(0.000553))
print(get_precision(0.100553))

In [ ]:
"""Some Nice Data Plots"""

fig, ax = plt.subplots(figsize=(10,10))
#plot x, y dependancy
ax.plot([-40000, 40000],[target, target])
#x='age', y='fdg', marker='.'
dfs = [test_s.dataframe[['DieX', 'DieY', target_param]].copy(), test_comp_s.dataframe[['DieX', 'DieY', target_param]].copy()]

colors= [['r', 'g'],['b', 'y']]
for ii, df in enumerate(dfs):
    """Plot 2D"""
    filt_3sigma(filt_positive(df, [1]), [0, 1, 2]).plot(x='DieX', y=target_param, ax=ax, kind='scatter', color=colors[ii][0])
    filt_3sigma(filt_positive(df, [2]), [0, 1, 2]).plot(x='DieY', y=target_param, ax=ax, kind='scatter', color=colors[ii][1])
    
"""Plot Wafer Maps"""

bins = 11
df1 = filt_3sigma(filt_positive(test_s.dataframe[['DieX', 'DieY', target_param]].copy(), [2]), [0, 1, 2])
df2 = filt_3sigma(filt_positive(test_comp_s.dataframe[['DieX', 'DieY', target_param]].copy(), [2]), [0, 1, 2])
    
vmin = round(np.min(df2[target_param]))
vmax = round(np.max(df1[target_param]))

print(vmin, vmax)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

plot_wafermap(ax1, df1['DieX'], df1['DieY'], df1[target_param], flatlocal=float(test_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax1.set_title('current')

plot_wafermap(ax2, df2['DieX'], df2['DieY'], df2[target_param], flatlocal=float(test_comp_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax2.set_title('previous')

df3 = filt_3sigma(filt_positive(test_s.dataframe[['DieX', 'DieY', 'diff_shift']].copy(), []), [0, 1, 2])
df4 = filt_3sigma(filt_positive(test_s.dataframe[['DieX', 'DieY', 'target_shift']].copy(), [2]), [0, 1, 2])
df5 = filt_3sigma(filt_positive(test_s.dataframe[['DieX', 'DieY', 'actual_shift']].copy(), [2]), [0, 1, 2])


vmin = round(min(np.min(df4['target_shift']), np.min(df5['actual_shift'])), 2)
vmax = round(max(np.max(df4['target_shift']), np.max(df5['actual_shift'])), 2)

f, ax = plt.subplots(figsize=(5,5))
plot_wafermap(ax, df3['DieX'], df3['DieY'], df3['diff_shift'], flatlocal=float(test_comp_s.header['FlatLocation']), bins=bins)
ax.set_title('Diffrence between actual and targeted shift')

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

plot_wafermap(ax1, df4['DieX'], df4['DieY'], df4['target_shift'], flatlocal=float(test_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax1.set_title('target')

plot_wafermap(ax2, df5['DieX'], df5['DieY'], df5['actual_shift'], flatlocal=float(test_comp_s.header['FlatLocation']), vmin=vmin, vmax=vmax, bins=bins)
ax2.set_title('actual')

#plot ibe
df = ibe_s.dataframe
f, ax = plt.subplots(figsize=(5,5))
plot_wafermap(ax, df['x']*1000, df['y']*1000, df['removal'], flatlocal=float(test_comp_s.header['FlatLocation']), bins=bins)
ax.set_title('ibe')

#display(filt_3sigma(filt_positive(test_s.dataframe[['DieNum', 'DieX', 'DieY', 'actual_shift']].copy(), [2]), [0, 1, 2]))

In [ ]:
"""Plot shift diff vs die x"""

yd = test_s.dataframe['diff_shift']
yb = yd<50
yd = yd[yb].reset_index(drop=True)
xd = test_s.dataframe['DieX'][yb].reset_index(drop=True)
xe = test_s.dataframe['DieY'][yb].reset_index(drop=True)
xc = test_s.dataframe['target_shift'][yb].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(10,10))
plot_n_fit(xd, yd, 'dieX', 'diff shift', ax)
fig, ax = plt.subplots(figsize=(10,10))
plot_n_fit(xe, yd, 'dieY', 'diff shift', ax)
fig, ax2 = plt.subplots(figsize=(10,10))
plot_n_fit(xc, yd, 'target shift', 'diff shift', ax2)

In [ ]:
"""Correlation of data"""

corr = test_s.dataframe.corr()

fig, ax = plt.subplots(figsize=(15,15))
im = ax.matshow(corr, cmap=plt.get_cmap('seismic'))
plt.colorbar(im)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)

display(corr)

In [ ]:
"""Plot Wafer Map"""
df = test_s.dataframe
x = df['DieX']
y = df['DieY']
z = df['ResFreq(S11)(1)']

xq, yq = mgrid[-50000:50000:100j,-50000:50000:100j]
points = np.array([-y.T,x.T])
z.shape

Z = griddata(points.T, z, (xq,yq), method='cubic')
#h = plt.pcolormesh(xq,yq,Z)
# Draw a Wafer
circle = plt.Circle((0,0), 50000, fill=False)

fig, ax = plt.subplots()
ax.add_artist(circle)

g = ax.contourf(xq,yq,Z)
plt.colorbar(g)
#plt.clabel(g, inline=1, fontsize=10)

In [ ]:
"""Plot Thickness Map"""
df = test_s.dataframe
x = df['Die x (mm)']
y = df['Die y (mm)']
z = df['Site 1 Layer 1 Thickness (A)']
gof = df['Site 1 GOF']

xq, yq = mgrid[-50:50:100j,-50:50:100j]
points = np.array([x,y])

Z = griddata(points.T, z, (xq,yq), method='cubic')

#h = plt.pcolormesh(xq,yq,Z)
# Draw a Wafer
circle = plt.Circle((0,0), 50, fill=False)

fig, ax = plt.subplots()
ax.add_artist(circle)

g = ax.contourf(xq,yq,Z)
plt.colorbar(g)
#plt.clabel(g, inline=1, fontsize=10)

In [ ]:
from IPython.core.display import HTML, display
#http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
display(HTML('''
<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Created with Jupyter, delivered by Fastly, rendered by Me.
</footer>'''))

In [ ]:
from IPython.core.display import HTML, display
#http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/
display(HTML('''
<script>
  $(document).ready(function(){
    $('div.prompt').show();
    $('div.back-to-top').show();
    $('nav#menubar').show();
    $('.breadcrumb').show();
    $('.hidden-print').show();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Created with Jupyter, delivered by Fastly, rendered by Me.
</footer>'''))